In [2]:
import numpy as np


In [1]:
import numpy as np
import os 
import cv2
collected_data_feedback_paht = 'data/new_collected_data_feedback.npy'
## load data 
data = np.load(collected_data_feedback_paht, allow_pickle=True).item()
filtered_data = {}
max_velocity = 0.2 
min_velocity = -0.2
for key, value in data.items():
    filtered_data[key] = value["tool_speed"] + [value["gripper_angle"]]
    filtered_data[key][3:6] = [0 for x in filtered_data[key][3:6]]
## normalize the filtered data based on max_velocity and min_velocity, max_velocity is 1, min_velocity is -1
for key, value in filtered_data.items():
    filtered_data[key][:-1] = [2 * ((x - min_velocity) / (max_velocity - min_velocity)) -1 for x in value[:-1]]
    if filtered_data[key][-1] < 0.5:
        filtered_data[key][-1] = -1
    else:
        filtered_data[key][-1] = 1
    
max_number = 0 
min_number = 0
for numbers in filtered_data.values():
    if max(numbers) > max_number:
        max_number = max(numbers)
    if min(numbers) < min_number:
        min_number = min(numbers)
        
print (max_number, min_number)
assert max_number <= 1 and min_number >= -1, "The data is not normalized"
episode_length = 50
## load image 
image_folder_path = 'data/pngs'
## load files in order
all_images = {}
for image_path in sorted(os.listdir(image_folder_path)):
    image = cv2.imread(os.path.join(image_folder_path, image_path))
    # image = cv2.resize(image, (224, 224))
    ## print file name 
    image_name = image_path.split('.')[0]
    all_images[image_name] = image

# assert len(filtered_data) == len(os.listdir(image_folder_path)), f"The number of images {len(filtered_data)} and data {len(os.listdir(image_folder_path))} is not equal"

for index in range(0, int(len(filtered_data)/episode_length)):
    episode = []
    for step in range(episode_length):
        data_name = list(filtered_data.keys())[index + step]
        episode.append({
            'image':all_images[data_name],
            'action': filtered_data[data_name],
            'language_instruction': 'lift carrot.',
        })
    if index < int(0.8 * (len(filtered_data)/episode_length)):
        np.save(f'data/training_data/episode_{index}', episode)
    else:
        np.save(f'data/val_data/episode_{index-int(0.8 * (len(filtered_data)/episode_length))}', episode)

    

    

    


0.9956331634521485 -0.6189485639333725
